## TabNet

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
#import pandas_profiling as pdp
import optuna
import mlflow

In [2]:
# 定数、フラグ関連
do_tuning_hypara = False
do_predict = False
do_submit = False
use_bestparam = False # optunaでtuningしたhyparaを使うか。
cv_n_splits = 10 
hypara_cv_n_splits = 5

In [3]:
trainpath =  './01_data/train.csv'
testpath =  './01_data/test.csv'

In [4]:
train_df = pd.read_csv(trainpath)
test_df = pd.read_csv(testpath)

## preprocessing

In [5]:
# dtypeの変換
train_df['Country'] = train_df['Country'].astype('category')
train_df['City'] = train_df['City'].astype('category')
train_df['year'] = train_df['year'].astype('category')

test_df['Country'] = test_df['Country'].astype('category')
test_df['City'] = test_df['City'].astype('category')
test_df['year'] = test_df['year'].astype('category')

In [6]:
def add_feature_common(df):
    # 北半球 or 南半球
    df['hemisphere'] = (df['lat'] >= 0)
    # 通算日の計算
    df['date'] = df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+df['day'].astype(str)
    df['date'] = pd.to_datetime(df['date'])
    # df['date_boy'] = pd.to_datetime(df['year'].astype(str)+'-01-01') # beginning of the year
    # df['totaldate'] = df['date'] - df['date_boy']
    df['totaldate'] = df['date'].dt.strftime("%j").astype(int)
    
    # 国別の平均CO中央値
    mean_co_mid = df.groupby('Country')['co_mid'].mean()
    df['country_average_co_mid'] = df['Country'].map(mean_co_mid).astype('float64')
    # 国別の最大CO中央値
    max_co_mid = df.groupby('Country')['co_mid'].max()
    df['country_max_co_mid'] = df['Country'].map(max_co_mid).astype('float64')
    # 国別の最小CO中央値
    min_co_mid = df.groupby('Country')['co_mid'].min()
    df['country_min_co_mid'] = df['Country'].map(min_co_mid).astype('float64')
    return df

def add_feature_train(df):
    # 国別の平均PM2.5中央値
    # 参考：https://teratail.com/questions/204630
    mean_pm25 = df.groupby('Country')['pm25_mid'].mean()
    df['country_average_pm25_mid'] = df['Country'].map(mean_pm25).astype('float64')
    # 国別の最大PM2.5中央値
    max_pm25 = df.groupby('Country')['pm25_mid'].max()
    df['country_max_pm25_mid'] = df['Country'].map(max_pm25).astype('float64')
    # 国別の最小PM2.5中央値
    min_pm25 = df.groupby('Country')['pm25_mid'].min()
    df['country_min_pm25_mid'] = df['Country'].map(min_pm25).astype('float64')
    
    return df
    
def add_feature_test(df):
    # 国別の平均PM2.5中央値
    mean_pm25 = train_df.groupby('Country')['pm25_mid'].mean()
    df['country_average_pm25_mid'] = df['Country'].map(mean_pm25).astype('float64')
    # 国別の最大PM2.5中央値
    max_pm25 = train_df.groupby('Country')['pm25_mid'].max()
    df['country_max_pm25_mid'] = df['Country'].map(max_pm25).astype('float64')
    # 国別の最小PM2.5中央値
    min_pm25 = train_df.groupby('Country')['pm25_mid'].min()
    df['country_min_pm25_mid'] = df['Country'].map(min_pm25).astype('float64')
    
    return df

In [7]:
train_df = add_feature_common(train_df)
train_df = add_feature_train(train_df)

test_df = add_feature_common(test_df)
test_df = add_feature_test(test_df)

In [8]:
# label encoding
def label_encoding(df):
    types = df.dtypes
    for col in df.columns:
        if types[col] == 'category':
            print(col)
            l_enc = LabelEncoder() # 本当はOne-Hotのほうがいい。
            df[col] = l_enc.fit_transform(df[col].values)
    return df

train_df = label_encoding(train_df)
test_df = label_encoding(test_df)

year
Country
City
year
Country
City


## train

In [9]:
features = [
#    'id',
#    'year',
#    'month',
#    'day',
    'Country',
#    'City',
    'lat',
    'lon',
    'co_cnt',
    'co_min',
    'co_mid',
    'co_max',
    'co_var',
    'o3_cnt',
    'o3_min',
    'o3_mid',
    'o3_max',
    'o3_var',
    'so2_cnt',
    'so2_min',
    'so2_mid',
    'so2_max',
    'so2_var',
    'no2_cnt',
    'no2_min',
    'no2_mid',
    'no2_max',
    'no2_var',
    'temperature_cnt',
    'temperature_min',
    'temperature_mid',
    'temperature_max',
    'temperature_var',
    'humidity_cnt',
    'humidity_min',
    'humidity_mid',
    'humidity_max',
    'humidity_var',
    'pressure_cnt',
    'pressure_min',
    'pressure_mid',
    'pressure_max',
    'pressure_var',
    'ws_cnt',
    'ws_min',
    'ws_mid',
    'ws_max',
    'ws_var',
    'dew_cnt',
    'dew_min',
    'dew_mid',
    'dew_max',
    'dew_var',
    #'pm25_mid',
    
    # additional feature
    'hemisphere', # 全く効いてない。
    'totaldate',
    'country_average_pm25_mid', # あまり効いてない。
    'country_max_pm25_mid',
    'country_min_pm25_mid',
    
    'country_max_co_mid',
    'country_min_co_mid',
]


In [10]:
X = train_df[features]
Y = train_df['pm25_mid']

In [11]:
# Hyper-parameter tuning
def opt(trial):
    '''
    hyper-parameter tuning with optuna.
    
    Ref:
        https://datadriven-rnd.com/lightgbm/
        https://kiroka-camp.com/lightgbm-optuna
        https://meknowledge.jpn.org/2021/05/28/lightgbm-optuna-tuning/
        https://knknkn.hatenablog.com/entry/2021/06/29/125226#learning_rate
        https://knknkn.hatenablog.com/entry/2021/06/14/160302
    '''
    pass

if do_tuning_hypara:
    pass
    

In [12]:
if use_bestparam:
    pass

In [28]:
x_train

array([[0, -27.46794, 153.02809, ..., 0.0, 56.111, 0.011],
       [0, -37.814, 144.96332, ..., 0.0, 56.111, 0.011],
       [0, -32.92953, 151.7801, ..., 0.0, 56.111, 0.011],
       ...,
       [28, 43.0389, -87.90647, ..., 2.593, 21.478, 0.012],
       [29, 21.0245, 105.84117, ..., 0.0, 20.961, 0.0],
       [29, 20.95045, 107.07336, ..., 0.0, 20.961, 0.0]], dtype=object)

In [35]:
models = []
cvscores = []
y_valid_dfs = []
y_valid_predict_dfs = []
vis_valid_df = pd.DataFrame(columns=['id', 'year', 'month', 'day', 'Country' ,'City', 'pm25_mid', 'pm25_mid_predict'])

gkf = GroupKFold(n_splits=cv_n_splits)
for n_fold, (train_idx, valid_idx) in enumerate(gkf.split(X, Y, groups=train_df['City'])):
    print('n_fold =', n_fold)
    
    if use_bestparam:
        model = TabNetRegressor()
        
    else:
        tabnet_params = {}
        model = TabNetRegressor()
    x_train, x_valid = X.iloc[train_idx].values, X.iloc[valid_idx].values
    y_train, y_valid = Y.iloc[train_idx].values, Y.iloc[valid_idx].values
    
    x_train = x_train.astype(np.float64)
    x_valid = x_valid.astype(np.float64)
    y_train = y_train.reshape(-1, 1).astype(np.float64)
    y_valid = y_valid.reshape(-1, 1).astype(np.float64)
    
#     print('type(x_train) =', type(x_train))
#     print('type(x_valid) =', type(x_valid))
#     print('type(y_train) =', type(y_train))
#     print('type(y_valid) =', type(y_valid))
    
#     print('x_train.shape =', x_train.shape)
#     print('x_valid.shape =', x_valid.shape)
#     print('y_train.shape =', y_train.shape)
#     print('y_valid.shape =', y_valid.shape)
    
    model.fit(
        x_train, y_train,
        eval_set = [(x_valid, y_valid)],
        eval_metric=['rmse'],
    )
    
    valid_df = pd.DataFrame(columns=['id', 'year', 'month', 'day', 'Country' ,'City', 'pm25_mid', 'pm25_mid_predict'])
    valid_df['id'] = train_df.loc[valid_idx, 'id']
    valid_df['year'] = train_df.loc[valid_idx, 'year']
    valid_df['month'] = train_df.loc[valid_idx, 'month']
    valid_df['day'] = train_df.loc[valid_idx, 'day']
    valid_df['Country'] = train_df.loc[valid_idx, 'Country']
    valid_df['City'] = train_df.loc[valid_idx, 'City']
        
    y_valid_predict = model.predict(x_valid)
    cvscore = np.sqrt(metrics.mean_squared_error(y_valid, y_valid_predict))
    valid_df['pm25_mid'] = y_valid
    valid_df['pm25_mid_predict'] = y_valid_predict    
    vis_valid_df = pd.concat([vis_valid_df, valid_df])
    models.append(model)
    cvscores.append(cvscore)

n_fold = 0
Device used : cpu
epoch 0  | loss: 1617.97238| val_0_rmse: 30.10752|  0:00:18s
epoch 1  | loss: 605.68247| val_0_rmse: 24.07431|  0:00:36s
epoch 2  | loss: 549.84449| val_0_rmse: 23.44275|  0:00:54s
epoch 3  | loss: 531.00323| val_0_rmse: 23.11697|  0:01:12s
epoch 4  | loss: 518.23374| val_0_rmse: 23.93003|  0:01:31s
epoch 5  | loss: 506.60005| val_0_rmse: 22.98071|  0:01:49s
epoch 6  | loss: 499.77104| val_0_rmse: 22.87582|  0:02:07s
epoch 7  | loss: 488.98211| val_0_rmse: 22.86712|  0:02:25s
epoch 8  | loss: 485.17918| val_0_rmse: 22.47601|  0:02:43s
epoch 9  | loss: 478.98132| val_0_rmse: 22.8474 |  0:03:05s
epoch 10 | loss: 477.16754| val_0_rmse: 22.47355|  0:03:26s
epoch 11 | loss: 473.73508| val_0_rmse: 22.57731|  0:04:09s
epoch 12 | loss: 470.52806| val_0_rmse: 22.53478|  0:04:56s
epoch 13 | loss: 467.5246| val_0_rmse: 22.58943|  0:05:57s
epoch 14 | loss: 466.46506| val_0_rmse: 22.74153|  0:06:22s
epoch 15 | loss: 465.26383| val_0_rmse: 22.33652|  0:06:43s
epoch 16 | 

In [36]:
average = np.array(cvscores).mean()

print('cvscores =', cvscores)
print('average =', average)


#average = 22.12377600122617 fold数= 20 learning_rate = 0.1
#average = 22.023999283119117 fold数= 10 learning_rate = 0.05 ←　ベースラインとする。
#average = 22.183219604203924 fold数= 20 learning_rate = 0.05
# average = 21.895404967428846 fold= 10 learning_rate = 0.05 ハイパラtuningあり。

cvscores = [22.204774247245762, 22.889636443211998, 25.10681190376925, 21.653486087867833, 21.89686928948803, 21.328252285201156, 22.249844708931448, 22.59259197693957, 21.446371275649913, 24.92200626911529]
average = 22.62906444874202


In [37]:
# feature importance
for model in models:
    lgb.plot_importance(model, figsize=(20, 10))
    plt.show()

TypeError: booster must be Booster or LGBMModel.

## validationの結果の可視化

In [38]:
def vis_result_varidation(Country, City):
    cond = (vis_valid_df['Country'] == Country) & (vis_valid_df['City'] == City)
    df = vis_valid_df[cond]
    
    fig, ax = plt.subplots(1, 1, figsize=(30, 8))
    ax.set_title(f'{Country}, {City}')
    ax.plot(df['pm25_mid'])
    ax.plot(df['pm25_mid_predict'])
    
    plt.show()

In [44]:
# vis_result_varidation('Australia', 'Brisbane')
# vis_result_varidation('China', 'Beijing')
# vis_result_varidation('Japan', 'Yokohama')

In [42]:
vis_valid_df

,id,year,month,day,Country,City,pm25_mid,pm25_mid_predict
1,2,0,1,1,0,39,13.741,23.691542
4,5,0,1,1,0,153,167.063,131.955231
25,26,0,1,1,6,69,171.113,135.718979
26,27,0,1,1,6,70,157.23,103.862152
34,35,0,1,1,6,135,68.843,90.847252
...,...,...,...,...,...,...,...,...
195885,195886,2,12,31,18,220,20.879,46.653111
195899,195900,2,12,31,22,155,33.818,53.077309
195924,195925,2,12,31,26,7,52.665,23.590929
195925,195926,2,12,31,26,46,152.706,145.070969


## predict

In [ ]:
y_pred = 0
if do_predict:
    y_pred = 0
    for i in range(len(models)):
        x_test = test_df[features]
        y_pred += models[i].predict(x_test)
    y_pred /= len(models)

## submit

In [ ]:
submission_df = pd.DataFrame(columns=['id', 'pm25_mid'])
submission_df['id'] = test_df['id'].copy()
submission_df['pm25_mid'] = y_pred
display(submission_df)

In [ ]:
if do_submit:
    save_filename = f'02_forSubmission/prediction_TabNet_No_{run_id}.csv'
    submission_df.to_csv(save_filename, header=False, index=False)
    